In [2]:
pip install tensorflow-addons

  Using cached tensorflow_addons-0.20.0-cp39-cp39-win_amd64.whl (746 kB)
  Using cached typeguard-2.13.3-py3-none-any.whl (17 kB)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

C:\Python39\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
# Load pre-trained ResNet50 model
resnet = keras.applications.ResNet50(include_top=False, weights="imagenet")

In [3]:
# Define the ViT model
input_shape = (224, 224, 3)
num_classes = 15

inputs = keras.Input(shape=input_shape)
x = layers.experimental.preprocessing.Rescaling(scale=1./255)(inputs)
x = layers.experimental.preprocessing.RandomFlip("horizontal")(x)
x = layers.experimental.preprocessing.RandomRotation(0.1)(x)
x = resnet(x, training=False)  # use ResNet50 as a feature extractor
x = layers.Flatten()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)

vit_classifier = keras.Model(inputs, outputs, name="vit_classifier")

In [4]:
# Define the optimizer and other hyperparameters
learning_rate = 1e-4
weight_decay = 1e-4
batch_size = 32
num_epochs = 15

optimizer = tfa.optimizers.AdamW(
    learning_rate=learning_rate, weight_decay=weight_decay
)

In [5]:
# Compile the model
vit_classifier.compile(
    optimizer=optimizer,
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[
        keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
        keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
    ],
)

In [6]:
# Prepare the data
train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    rotation_range=0.1,
    validation_split=0.1
)

train_generator = train_datagen.flow_from_directory(
    "split_data/train",
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode="sparse",
    shuffle=True,
    subset="training",
)

val_generator = train_datagen.flow_from_directory(
    "split_data/train",
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode="sparse",
    shuffle=False,
    subset="validation",
)

test_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
)

test_generator = test_datagen.flow_from_directory(
    "split_data/test",
    target_size=input_shape[:2],
    batch_size=batch_size,
    class_mode="sparse",
    shuffle=False,
)

Found 4991 images belonging to 15 classes.
Found 545 images belonging to 15 classes.
Found 706 images belonging to 15 classes.


In [7]:
import os
from tensorflow import keras

# Create a directory for the model checkpoint
checkpoint_dir = "./my_checkpoint_directory"
os.makedirs(checkpoint_dir, exist_ok=True)

# Set the checkpoint file path
checkpoint_filepath = os.path.join(checkpoint_dir, "checkpoint")

# Create a ModelCheckpoint callback to save the best weights
checkpoint_callback = keras.callbacks.ModelCheckpoint(
    checkpoint_filepath,
    monitor="val_accuracy",
    save_best_only=True,
    save_weights_only=True,
)

# Train the model with the checkpoint callback
history = vit_classifier.fit(
    train_generator,
    epochs=num_epochs,
    validation_data=val_generator,
    callbacks=[checkpoint_callback],
)


Epoch 1/15


C:\Python39\lib\site-packages\keras\backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


156/156 [==============================] - 1466s 9s/step - loss: 2.4739 - accuracy: 0.1573 - top-5-accuracy: 0.6241 - val_loss: 2.3551 - val_accuracy: 0.1927 - val_top-5-accuracy: 0.6789
Epoch 2/15
156/156 [==============================] - 1381s 9s/step - loss: 2.4322 - accuracy: 0.1631 - top-5-accuracy: 0.6458 - val_loss: 2.3436 - val_accuracy: 0.1927 - val_top-5-accuracy: 0.6771
Epoch 3/15
156/156 [==============================] - 1348s 9s/step - loss: 2.4102 - accuracy: 0.1699 - top-5-accuracy: 0.6560 - val_loss: 2.3487 - val_accuracy: 0.1927 - val_top-5-accuracy: 0.6771
Epoch 4/15
156/156 [==============================] - 1341s 9s/step - loss: 2.4086 - accuracy: 0.1771 - top-5-accuracy: 0.6562 - val_loss: 2.3708 - val_accuracy: 0.1927 - val_top-5-accuracy: 0.6771
Epoch 5/15
156/156 [==============================] - 1349s 9s/step - loss: 2.4017 - accuracy: 0.1749 - top-5-accuracy: 0.6542 - val_loss: 2.3324 - val_accuracy: 0.1927 - val_top-5-accuracy: 0.6771
Epoch 6/15
156/156 [=

In [8]:
# Evaluate the model epoc 15 
vit_classifier.load_weights(checkpoint_filepath)
_, accuracy, top_5_accuracy = vit_classifier.evaluate(test_generator)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")
print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

23/23 [==============================] - 56s 2s/step - loss: 2.3864 - accuracy: 0.1884 - top-5-accuracy: 0.6657
Test accuracy: 18.84%
Test top 5 accuracy: 66.57%


In [11]:
# Evaluate the model
vit_classifier.load_weights(checkpoint_filepath)
_, accuracy, top_5_accuracy = vit_classifier.evaluate(test_generator)
print(f"Test accuracy: {round(accuracy * 100, 2)}%")
print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

23/23 [==============================] - 41s 2s/step - loss: 2.3775 - accuracy: 0.1884 - top-5-accuracy: 0.6643
Test accuracy: 18.84%
Test top 5 accuracy: 66.43%


In [12]:
# Load the best weights from the model checkpoint
vit_classifier.load_weights(checkpoint_filepath)

# Evaluate the model on the test dataset
test_loss, test_accuracy = vit_classifier.evaluate(test_generator)

print("Test loss:", test_loss)
print("Test accuracy:", test_accuracy)


23/23 [==============================] - 41s 2s/step - loss: 2.3775 - accuracy: 0.1884 - top-5-accuracy: 0.6643


ValueError: too many values to unpack (expected 2)